In [1]:
import os
import numpy as np
import tensorflow as tf

/home/dof6/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
training_writer = tf.python_io.TFRecordWriter('./grasp_data/t_shape/training.tfrecord')
test_writer = tf.python_io.TFRecordWriter('./grasp_data/t_shape/test.tfrecord')
            
ratio = 0.8
training_positive = 0
training_negative = 0
test_positive = 0
test_negative = 0
for file_name in os.listdir('./grasp_data/t_shape/'):
    if ".npy" in file_name:
        data = np.load('./grasp_data/t_shape/'+file_name,allow_pickle=True)[()]
        
        grasp_quality_list = np.asarray(data['grasp_quality_list']).squeeze()
        eef_position_list = np.asarray(data['eef_position_list']).squeeze()
        eef_quaternion_list = np.asarray(data['eef_quaternion_list']).squeeze()
        eef_euler_list = np.asarray(data['eef_euler_list']).squeeze()
        cropped_color_image_list = np.asarray(data['cropped_color_image_list']).squeeze()
        cropped_depth_image_list = np.asarray(data['cropped_depth_image_list']).squeeze()
        grasp_width_list = np.asarray(data['grasp_width_list']).squeeze()
        grasp_depth_list = np.asarray(data['grasp_depth_list']).squeeze()

        for grasp_quality, eef_position, eef_quaternion, eef_euler, cropped_color_image, cropped_depth_image, grasp_width, grasp_depth in zip(grasp_quality_list, eef_position_list, eef_quaternion_list, eef_euler_list, cropped_color_image_list, cropped_depth_image_list, grasp_width_list, grasp_depth_list):
            cropped_depth_image = cropped_depth_image[:,:,np.newaxis]
            example = tf.train.Example(features=tf.train.Features(feature={
                'eef_position': tf.train.Feature(bytes_list=tf.train.BytesList(value=[eef_position.astype(np.float32).tostring()])),
                'eef_quaternion': tf.train.Feature(bytes_list=tf.train.BytesList(value=[eef_quaternion.astype(np.float32).tostring()])),
                'eef_euler': tf.train.Feature(bytes_list=tf.train.BytesList(value=[eef_euler.astype(np.float32).tostring()])),
                'cropped_color_image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[cropped_color_image.astype(np.float32).tostring()])),
                'cropped_depth_image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[cropped_depth_image.astype(np.float32).tostring()])),
                'grasp_width': tf.train.Feature(float_list=tf.train.FloatList(value=[grasp_width.astype(np.float32)])),
                'grasp_depth': tf.train.Feature(float_list=tf.train.FloatList(value=[grasp_depth.astype(np.float32)])),
                'grasp_quality': tf.train.Feature(float_list=tf.train.FloatList(value=[grasp_quality.astype(np.float32)]))
            }))
            if np.random.uniform() < ratio:
                training_writer.write(example.SerializeToString())
                if grasp_quality == 0.0:
                    training_negative+=1
                else:
                    training_positive+=1
            else:
                test_writer.write(example.SerializeToString())
                if grasp_quality == 0.0:
                    test_negative+=1
                else:
                    test_positive+=1
                
print(grasp_quality.shape)
print(grasp_quality.dtype)
print(eef_position.shape)
print(eef_position.dtype)
print(eef_quaternion.shape)
print(eef_quaternion.dtype)
print(cropped_color_image.shape)
print(cropped_color_image.dtype)
print(cropped_depth_image.shape)
print(cropped_depth_image.dtype)
print(grasp_width.shape)
print(grasp_width.dtype)
print(grasp_depth.shape)
print(grasp_depth.dtype)

print(training_positive)
print(training_negative)
print(test_positive)
print(test_negative)

training_writer.close()
test_writer.close()

()
float64
(3,)
float64
(4,)
float64
(50, 50, 3)
float32
(50, 50, 1)
float32
()
float64
()
float64
21403
95863
5377
24137


In [2]:
def _parse_function(example_proto):
    features = {
        'eef_position': tf.FixedLenFeature([], tf.string),
        'eef_quaternion': tf.FixedLenFeature([], tf.string),
        'eef_euler': tf.FixedLenFeature([], tf.string),
        'cropped_color_image': tf.FixedLenFeature([], tf.string),
        'cropped_depth_image': tf.FixedLenFeature([], tf.string),
        'grasp_width': tf.FixedLenFeature([], tf.float32),
        'grasp_depth': tf.FixedLenFeature([], tf.float32),
        'grasp_quality': tf.FixedLenFeature([], tf.float32)
    }
    parsed_features = tf.parse_single_example(example_proto, features)
    
    gripper_position = tf.decode_raw((parsed_features['eef_position']), tf.float32)
    gripper_position = tf.reshape(gripper_position,[3])
    
    gripper_quaternion = tf.decode_raw((parsed_features['eef_quaternion']), tf.float32)
    gripper_quaternion = tf.reshape(gripper_quaternion,[4])
    
    gripper_euler = tf.decode_raw((parsed_features['eef_euler']), tf.float32)
    gripper_euler = tf.reshape(gripper_euler,[3])
    
    cropped_color_image = tf.decode_raw((parsed_features['cropped_color_image']), tf.float32)
    cropped_color_image = tf.reshape(cropped_color_image,[50, 50, 3])
    
    cropped_depth_image = tf.decode_raw((parsed_features['cropped_depth_image']), tf.float32)
    cropped_depth_image = tf.reshape(cropped_depth_image,[50, 50, 1])
    
    grasp_depth = parsed_features['grasp_depth']
    grasp_depth = tf.reshape(grasp_depth,[1])
    
    grasp_width = parsed_features['grasp_width']
    grasp_width = tf.reshape(grasp_width,[1])
    
    grasp_quality = parsed_features['grasp_quality']
    grasp_quality = tf.reshape(grasp_quality,[1])

    return gripper_position, gripper_quaternion, gripper_euler, cropped_color_image, cropped_depth_image, grasp_depth, grasp_width, grasp_quality

In [3]:
tf.reset_default_graph()
training_dataset = tf.data.TFRecordDataset('./grasp_data/t_shape/training.tfrecord')
training_dataset = training_dataset.map(_parse_function)

training_dataset = training_dataset.repeat()
training_dataset = training_dataset.batch(1)
training_dataset = training_dataset.shuffle(buffer_size=1)

training_iterator = training_dataset.make_initializable_iterator()
training_gripper_position, training_gripper_quaternion, training_gripper_euler, training_cropped_color_image, training_cropped_depth_image, training_grasp_depth, training_grasp_width, training_grasp_quality = training_iterator.get_next()

print(training_gripper_position.shape)
print(training_gripper_quaternion.shape)
print(training_cropped_color_image.shape)
print(training_cropped_depth_image.shape)
print(training_grasp_quality.shape)

W0803 14:05:49.975647 140365690476288 deprecation.py:323] From <ipython-input-3-621945151005>:9: make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


(?, 3)
(?, 4)
(?, 50, 50, 3)
(?, 50, 50, 1)
(?, 1)


In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(training_iterator.initializer)

print(sess.run(training_gripper_position))
print(sess.run(training_gripper_quaternion))
print(sess.run(training_cropped_color_image))
print(sess.run(training_cropped_depth_image))
print(sess.run(training_grasp_quality))

[[-0.00941823  0.00907065 -0.928466  ]]
[[0.96286094 0.11226846 0.08435628 0.23060487]]
[[[[0.5294118  0.29803923 0.07843138]
   [0.5137255  0.28627452 0.07450981]
   [0.5137255  0.28627452 0.07450981]
   ...
   [0.5372549  0.32156864 0.10196079]
   [0.5372549  0.32156864 0.10196079]
   [0.5372549  0.32156864 0.10196079]]

  [[0.5294118  0.29803923 0.07843138]
   [0.5137255  0.28627452 0.07450981]
   [0.5137255  0.28627452 0.07450981]
   ...
   [0.5372549  0.32156864 0.10196079]
   [0.5372549  0.32156864 0.10196079]
   [0.7176471  0.16078432 0.05098039]]

  [[0.54509807 0.3137255  0.09019608]
   [0.5411765  0.30980393 0.09019608]
   [0.5411765  0.30980393 0.09019608]
   ...
   [0.69803923 0.13725491 0.03921569]
   [0.9019608  0.         0.        ]
   [0.9019608  0.         0.        ]]

  ...

  [[0.64705884 0.44705883 0.24705882]
   [0.67058825 0.46666667 0.2627451 ]
   [0.67058825 0.46666667 0.2627451 ]
   ...
   [0.5803922  0.37254903 0.16078432]
   [0.5803922  0.37254903 0.1607843